In [1]:
! pip install Faker


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 1.2 MB/s eta 0:00:00 MB/s eta 0:00:01:010m
  Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [4]:
pip install apache-flink


  Using cached apache_flink-2.1.0-cp39-cp39-macosx_11_0_arm64.whl (2.5 MB)
  Using cached ruamel.yaml-0.18.15-py3-none-any.whl (119 kB)
  Using cached apache_beam-2.61.0-cp39-cp39-macosx_15_0_arm64.whl
  Using cached avro-1.12.0-py2.py3-none-any.whl (124 kB)
  Using cached protobuf-6.32.1-cp39-abi3-macosx_10_9_universal2.whl (426 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Using cached pyarrow-20.0.0-cp39-cp39-macosx_12_0_arm64.whl (30.8 MB)
  Using cached pandas-2.3.2-cp39-cp39-macosx_11_0_arm64.whl (10.8 MB)
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Using cached pemja-0.5.3-cp39-cp39-macosx_11_0_arm64.whl (53 kB)
  Using cached httplib2-0.31.0-py3-none-any.whl (91 kB)
  Using cached apache_flink_libraries-2.1.0.tar.gz (309.7 MB)
  Preparing metadata (setup.py) ... done
  Using cached fastavro-1.12.0-cp39-cp39-macosx_10_9_universal2.whl (950 kB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl (20 kB)
  Using cached numpy-2.0.2-cp39-cp39-macosx_1

In [3]:
! java -version

openjdk version "17.0.15" 2025-04-15
OpenJDK Runtime Environment Homebrew (build 17.0.15+0)
OpenJDK 64-Bit Server VM Homebrew (build 17.0.15+0, mixed mode, sharing)


In [5]:
from pyflink.table import EnvironmentSettings, TableEnvironment
from faker import Faker


In [6]:
env_settings = EnvironmentSettings.in_batch_mode()
table_env = TableEnvironment.create(env_settings) 

In [11]:
from pyflink.table.expressions import col 

In [7]:
fake = Faker()

data = [(fake.name(), fake.city(), fake.state()) for _ in range(10)]  # Generate 10 rows of fake data

column_names = ["name", "city", "state"]


In [8]:
table = table_env.from_elements(data, schema=column_names)

# Print the table
table.execute().print() 

+--------------------------------+--------------------------------+--------------------------------+
|                           name |                           city |                          state |
+--------------------------------+--------------------------------+--------------------------------+
|                    Harold Ruiz |                    Vazquezview |                   North Dakota |
|                    Joshua Gray |                    Lake Daniel |                       New York |
|               Austin Patterson |                    Garrettside |                    Connecticut |
|                   Alan Spencer |              North Gregoryberg |                       Virginia |
|                      Robyn Lee |              West Valerieshire |                           Iowa |
|                      Adam York |                 South Tammyton |                       Arkansas |
|                    Henry Davis |             New Claudiachester |                        

In [9]:
table_env.create_temporary_view('source_table', table)

table_env.execute_sql(f"SELECT * FROM source_table ").print()


+--------------------------------+--------------------------------+--------------------------------+
|                           name |                           city |                          state |
+--------------------------------+--------------------------------+--------------------------------+
|                    Harold Ruiz |                    Vazquezview |                   North Dakota |
|                    Joshua Gray |                    Lake Daniel |                       New York |
|               Austin Patterson |                    Garrettside |                    Connecticut |
|                   Alan Spencer |              North Gregoryberg |                       Virginia |
|                      Robyn Lee |              West Valerieshire |                           Iowa |
|                      Adam York |                 South Tammyton |                       Arkansas |
|                    Henry Davis |             New Claudiachester |                        

In [12]:
table.select(col("name"), col("city")).execute().print() 

+--------------------------------+--------------------------------+
|                           name |                           city |
+--------------------------------+--------------------------------+
|                    Harold Ruiz |                    Vazquezview |
|                    Joshua Gray |                    Lake Daniel |
|               Austin Patterson |                    Garrettside |
|                   Alan Spencer |              North Gregoryberg |
|                      Robyn Lee |              West Valerieshire |
|                      Adam York |                 South Tammyton |
|                    Henry Davis |             New Claudiachester |
|                Michele Compton |                     New Eugene |
|                    John Wright |               North Katieshire |
|                     Becky Rios |                      Karenstad |
+--------------------------------+--------------------------------+
10 rows in set


In [13]:
from pyflink.table.expressions import col

table \
    .select(col("name"), col("city"), col("state")) \
    .where(col("state") == 'Vermont') \
    .execute().print() 

Empty set


In [14]:
table \
    .group_by(col("state")) \
    .select(col("state").alias("state"), col("name").count.alias("count")) \
    .execute().print()

+--------------------------------+----------------------+
|                          state |                count |
+--------------------------------+----------------------+
|                   South Dakota |                    1 |
|                           Utah |                    1 |
|                  West Virginia |                    1 |
|                   North Dakota |                    1 |
|                    Connecticut |                    1 |
|                         Kansas |                    1 |
|                       Arkansas |                    1 |
|                       Virginia |                    1 |
|                           Iowa |                    1 |
|                       New York |                    1 |
+--------------------------------+----------------------+
10 rows in set


In [15]:
table_env.execute_sql("""
    CREATE TABLE print_sink (
        name STRING, 
        city STRING,
        state STRING
    ) WITH (
        'connector' = 'print'
    )
""")

table_env.execute_sql("""
    INSERT INTO print_sink
        SELECT * FROM source_table
""").wait() 

1> +I[Harold Ruiz, Vazquezview, North Dakota]
1> +I[Joshua Gray, Lake Daniel, New York]
1> +I[Austin Patterson, Garrettside, Connecticut]
1> +I[Alan Spencer, North Gregoryberg, Virginia]
1> +I[Robyn Lee, West Valerieshire, Iowa]
1> +I[Adam York, South Tammyton, Arkansas]
1> +I[Henry Davis, New Claudiachester, Utah]
1> +I[Michele Compton, New Eugene, South Dakota]
1> +I[John Wright, North Katieshire, West Virginia]
1> +I[Becky Rios, Karenstad, Kansas]


In [16]:
table_result = table_env.execute_sql(f"SELECT * FROM source_table ")

with table_result.collect() as results:
   for result in results:
       print(result) 

<Row('Harold Ruiz', 'Vazquezview', 'North Dakota')>
<Row('Joshua Gray', 'Lake Daniel', 'New York')>
<Row('Austin Patterson', 'Garrettside', 'Connecticut')>
<Row('Alan Spencer', 'North Gregoryberg', 'Virginia')>
<Row('Robyn Lee', 'West Valerieshire', 'Iowa')>
<Row('Adam York', 'South Tammyton', 'Arkansas')>
<Row('Henry Davis', 'New Claudiachester', 'Utah')>
<Row('Michele Compton', 'New Eugene', 'South Dakota')>
<Row('John Wright', 'North Katieshire', 'West Virginia')>
<Row('Becky Rios', 'Karenstad', 'Kansas')>


In [17]:
pandas_df = table.to_pandas()
pandas_df


,name,city,state
0,Harold Ruiz,Vazquezview,North Dakota
1,Joshua Gray,Lake Daniel,New York
2,Austin Patterson,Garrettside,Connecticut
3,Alan Spencer,North Gregoryberg,Virginia
4,Robyn Lee,West Valerieshire,Iowa
5,Adam York,South Tammyton,Arkansas
6,Henry Davis,New Claudiachester,Utah
7,Michele Compton,New Eugene,South Dakota
8,John Wright,North Katieshire,West Virginia
9,Becky Rios,Karenstad,Kansas


In [19]:
table_temp = table_env.from_pandas(pandas_df)
table_temp.execute().print() 

+--------------------------------+--------------------------------+--------------------------------+
|                           name |                           city |                          state |
+--------------------------------+--------------------------------+--------------------------------+
|                    Harold Ruiz |                    Vazquezview |                   North Dakota |
|                    Joshua Gray |                    Lake Daniel |                       New York |
|               Austin Patterson |                    Garrettside |                    Connecticut |
|                   Alan Spencer |              North Gregoryberg |                       Virginia |
|                      Robyn Lee |              West Valerieshire |                           Iowa |
|                      Adam York |                 South Tammyton |                       Arkansas |
|                    Henry Davis |             New Claudiachester |                        

In [20]:
table.execute().print()


+--------------------------------+--------------------------------+--------------------------------+
|                           name |                           city |                          state |
+--------------------------------+--------------------------------+--------------------------------+
|                    Harold Ruiz |                    Vazquezview |                   North Dakota |
|                    Joshua Gray |                    Lake Daniel |                       New York |
|               Austin Patterson |                    Garrettside |                    Connecticut |
|                   Alan Spencer |              North Gregoryberg |                       Virginia |
|                      Robyn Lee |              West Valerieshire |                           Iowa |
|                      Adam York |                 South Tammyton |                       Arkansas |
|                    Henry Davis |             New Claudiachester |                        